In [ ]:
import sqlite3

import pandas as pd

In [ ]:
with open("table.sql", "r") as file:
    sql_content = file.read()

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Execute the SQL from the file
cursor.executescript(sql_content)

# Query the data back into a DataFrame
df = pd.read_sql_query("SELECT * FROM transactions", conn)
df["transaction_time"] = pd.to_datetime(df["transaction_time"])

print("Transactions DataFrame from table.sql:")
print(f"Shape: {df.shape}")
print(df.head(10))
print(f"\nDate range: {df['transaction_time'].min()} to {df['transaction_time'].max()}")

Transactions DataFrame from table.sql:
Shape: (114, 2)
     transaction_time  transaction_amount
0 2021-01-16 00:05:54               25.05
1 2021-01-07 20:53:04              124.00
2 2021-01-18 22:55:37               66.58
3 2021-01-21 00:36:57                9.99
4 2021-01-19 06:31:10               22.27
5 2021-01-10 01:24:04              576.76
6 2021-01-04 00:07:27               49.91
7 2021-01-25 20:36:17               14.11
8 2021-01-08 21:11:16              112.21
9 2021-01-07 00:06:21              331.80

Date range: 2021-01-01 00:12:28 to 2021-01-31 13:00:35


In [ ]:
# Calculate January 31's rolling 3-day average of total transaction amount processed per day
# Extract date and group by day to get daily totals
df["date"] = df["transaction_time"].dt.date
daily_totals = df.groupby("date")["transaction_amount"].sum().reset_index()

# Convert to pandas DataFrame with date as index for rolling calculation
daily_totals_df = pd.DataFrame(daily_totals)
daily_totals_df["date"] = pd.to_datetime(daily_totals_df["date"])
daily_totals_df = daily_totals_df.set_index("date").sort_index()

# Calculate 3-day rolling average
daily_totals_df["rolling_3day_avg"] = (
    daily_totals_df["transaction_amount"].rolling(window=3, min_periods=1).mean()
)

print("\nDaily totals with 3-day rolling average (last 10 days):")
print(daily_totals_df.tail(10))

# Find January 31st rolling average
jan_31_2021 = pd.to_datetime("2021-01-31")
if jan_31_2021 in daily_totals_df.index:
    print(f"\tJanuary 31, 2021:")
    jan_31_avg = daily_totals_df.loc[jan_31_2021, "rolling_3day_avg"]
    jan_31_total = daily_totals_df.loc[jan_31_2021, "transaction_amount"]
    print(f"\t\tDaily total: ${jan_31_total:.2f}")
    print(f"\t\tRolling 3-day average: ${jan_31_avg:.2f}")
else:
    print(f"\nJanuary 31, 2021 not found in data.")

conn.close()


Daily totals with 3-day rolling average (last 10 days):
            transaction_amount  rolling_3day_avg
date                                            
2021-01-22              527.00        434.326667
2021-01-23             1226.92        793.296667
2021-01-24             1165.30        973.073333
2021-01-25              290.95        894.390000
2021-01-26             3004.47       1486.906667
2021-01-27              471.34       1255.586667
2021-01-28             8974.41       4150.073333
2021-01-29             1520.90       3655.550000
2021-01-30              466.12       3653.810000
2021-01-31               59.43        682.150000
	January 31, 2021:
		Daily total: $59.43
		Rolling 3-day average: $682.15


## Answer
- January 31, 2021:
  - Daily total: $59.43
  - Rolling 3-day average: $682.15